In [51]:
#pip install pyarrow
#pip install fsspec
#pip install huggingface_hub

In [59]:
import pandas as pd
import math
import numpy as np

splits = {'train': 'plain_text/train-00000-of-00001.parquet', 'test': 'plain_text/test-00000-of-00001.parquet', 'unsupervised': 'plain_text/unsupervised-00000-of-00001.parquet'}
dfC = pd.read_parquet("hf://datasets/stanfordnlp/imdb/" + splits["train"])

In [41]:
print(len(dfC))
df= df.sample(100)
print(len(df))
df= df.reset_index(drop=True)

25000
100


In [42]:
print(df['text'][0])
print(df.columns)

Charles Bronson continued his 80's slump with what could possibly be his worst movie of the decade, if not ever. In it he plays a detective who goes to great lengths to track down a pimp who has abducted an Asian girl and is currently selling her to the highest bidder. It's a shameful attempt at action and is quite possibly the most depressing movie ever made. A lot of people consider the Death Wish sequels to be Bronson's worst movies. The Death Wish sequels weren't great movies but were in most cases competently made and very watchable. Kinjite however is an utterly dull thriller which is punctuated by mean spirited racism and child molestation. It really is a rephrensible film and all of it is made worse by the fact that it tries to preach a "Law and Justice" mentality that is too overwrought to really say anything. Indeed the movie begins with underage sex being interrupted with Bronson then anally raping the pedophile with a dildo. The Chicago theater I watched this in just gasped

In [69]:
CP=0
CN=0
for x in df['label']:
    if x==0:
        CN+=1
    else:
        CP+=1
print(CN)
print(CP)
PP=CP/len(df)
PN=CN/len(df)
print(PP,PN)





c = [0,0]
for i,sample in df.iterrows():
  c[sample['label']] += 1
logP = [math.log10(c[l]/len(df)) for l in range(2)]
print([10**p for p in logP])

58
42
0.42 0.58
[0.58, 0.42]


In [49]:
import nltk
#nltk.download('punkt_tab')
#nltk.download('punkt')
from nltk.tokenize import word_tokenize

V = set()
docs = [[],[]]

for i,sample in df.iterrows():
  tokens = word_tokenize(sample['text'])
  V = V.union(set(tokens))
  docs[sample['label']].extend(tokens) # extend per evitare liste annidate



In [57]:
print(len(docs[1]))

11680


In [102]:
count_w_c = [{w:0 for w in V} for c in range(2)]
count_c = [0,0]
for c in [0,1]:
  for w in docs[c]:
      count_w_c[c][w] += 1

logP_w_c = [{w : math.log10((count_w_c[c][w]+1)/(len(docs[c])+len(V))) for w in V} for c in [0,1]]

In [61]:
print("awful", 10**logP_w_c[0]['awful'], 10**logP_w_c[1]['awful'])
print("great", 10**logP_w_c[0]['great'], 10**logP_w_c[1]['great'])
print("horrible", 10**logP_w_c[0]['horrible'], 10**logP_w_c[1]['horrible'])
print("fantastic", 10**logP_w_c[0]['fantastic'], 10**logP_w_c[1]['fantastic'])

awful 0.00046886321981160216 0.00011576753878212549
great 0.0005114871488853842 0.0009261403102570042
horrible 0.00017049571629512816 5.788376939106279e-05
fantastic 4.262392907378203e-05 0.00017365130817318826


In [70]:
def classify(sentence):
  words = word_tokenize(sentence)
  logP_sentence = [logP[c] + sum([logP_w_c[c][w] for w in words if w in V]) for c in [0,1]]
  return np.argmax(logP_sentence)

In [72]:
sentence = "this is awful movie"
print(sentence + ". Class: " + str(classify(sentence)))
sentence = "this is a horrible movie"
print(sentence + ". Class: " + str(classify(sentence)))
sentence = "this is a fantastic movie"
print(sentence + ". Class: " + str(classify(sentence)))
sentence = "this is a stupid movie"
print(sentence + ". Class: " + str(classify(sentence)))

this is awful movie. Class: 0
this is a horrible movie. Class: 0
this is a fantastic movie. Class: 1
this is a stupid movie. Class: 1
